# Sign Language Classification

Input: 64 x 64 x 1 Images (.npy)

Output: 0~9 Number

Used CNN Model.

1. Library Importing

In [0]:
# if not torch: install()

# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np

from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

2. Device Configuration, Hyper-Params Setting

In [0]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 10
batch_size = 100
learning_rate = 0.001

3. Model Building (CNN)

In [0]:
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()
    
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=9)
    self.conv2 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=7)
    self.fc1 = nn.Linear(11 * 11 * 6, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
   
  
  def forward(self, x):
    out = F.relu(self.conv1(x))  # None * 3 * 56 * 56
    out = F.max_pool2d(out, 2)  # None * 3 * 28 * 28
    out = F.relu(self.conv2(out))  # None * 6 * 22 * 22
    out = F.max_pool2d(out, 2)  # None * 6 * 11 * 11
    out = out.view(out.size(0), -1)
    out = F.relu(self.fc1(out))
    out = F.relu(self.fc2(out))
    out = self.fc3(out)
    return out

4. Custom Dataset

In [0]:
class SignlangDataset(torch.utils.data.Dataset):
  def __init__(self, X, Y, onehot=True):
    X = X.reshape((-1, 1, 64, 64))
    if onehot:
      Y = np.argmax(Y, axis=1)
    
    self.X = torch.from_numpy(X)
    self.Y = torch.from_numpy(Y)
  
  
  def __getitem__(self, index):
    X = self.X[index]
    Y = self.Y[index]
    sample = {'X': X, 'Y': Y}
    return sample
  
  
  def __len__(self):
    return len(self.Y)

5. Data Loading

In [0]:
X = np.load('X.npy')
Y = np.load('Y.npy')
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

trainSignData = SignlangDataset(x_train, y_train, onehot=True)
trainDataLoader = DataLoader(trainSignData, shuffle=True, batch_size=32)

testSignData = SignlangDataset(x_test, y_test, onehot=True)
testDataLoader = DataLoader(testSignData, shuffle=False, batch_size=32)

6. Defining Nets

In [0]:
net = NeuralNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
best_acc = 0
start_epoch = 0

7. Defining Train Method

In [0]:
def train(num_epochs):
  net.train()
  train_loss = correct = total = 0
  for batch_index, sample in enumerate(trainDataLoader):
    optimizer.zero_grad()
    inputs, labels = Variable(sample['X']), Variable(sample['Y'])
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    train_loss += loss
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += predicted.eq(labels.data).cpu().sum()
    
    print('TRAIN Epoch %d | Loss: %.3f | Acc: %.3f%% (%d / %d)'
          % (num_epochs, train_loss / (batch_index + 1), 100. * correct / total,
            correct, total))

8. Defining Test Method

In [0]:
def test(num_epochs):
  net.eval()
  test_loss = correct = total = 0
  for batch_index, sample in enumerate(testDataLoader):
    inputs, labels = Variable(sample['X'], volatile=True), Variable(sample['Y'])
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    
    test_loss += loss
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += predicted.eq(labels.data).cpu().sum()
    print('TEST Epoch %d | Loss: %.3f | Acc: %.3f%% (%d / %d)'
         % (num_epochs, test_loss / (batch_index + 1), 100. * correct / total,
           correct, total))

9. Start Learning!

In [19]:
for epoch in range(start_epoch, start_epoch + 15):
  train(epoch)
  test(epoch)

TRAIN Epoch 0 | Loss: 1.312 | Acc: 37.500% (12 / 32)
TRAIN Epoch 0 | Loss: 1.202 | Acc: 46.875% (30 / 64)
TRAIN Epoch 0 | Loss: 1.178 | Acc: 48.958% (47 / 96)
TRAIN Epoch 0 | Loss: 1.173 | Acc: 50.000% (64 / 128)
TRAIN Epoch 0 | Loss: 1.228 | Acc: 45.625% (73 / 160)
TRAIN Epoch 0 | Loss: 1.251 | Acc: 42.708% (82 / 192)
TRAIN Epoch 0 | Loss: 1.219 | Acc: 45.089% (101 / 224)
TRAIN Epoch 0 | Loss: 1.236 | Acc: 44.531% (114 / 256)
TRAIN Epoch 0 | Loss: 1.271 | Acc: 43.403% (125 / 288)
TRAIN Epoch 0 | Loss: 1.273 | Acc: 43.125% (138 / 320)
TRAIN Epoch 0 | Loss: 1.282 | Acc: 41.477% (146 / 352)
TRAIN Epoch 0 | Loss: 1.300 | Acc: 41.927% (161 / 384)
TRAIN Epoch 0 | Loss: 1.310 | Acc: 41.587% (173 / 416)
TRAIN Epoch 0 | Loss: 1.299 | Acc: 41.741% (187 / 448)
TRAIN Epoch 0 | Loss: 1.281 | Acc: 43.125% (207 / 480)
TRAIN Epoch 0 | Loss: 1.288 | Acc: 42.578% (218 / 512)
TRAIN Epoch 0 | Loss: 1.296 | Acc: 42.647% (232 / 544)
TRAIN Epoch 0 | Loss: 1.304 | Acc: 42.361% (244 / 576)
TRAIN Epoch 0 | Los